In [1]:
# IMPORT Thư viện
#  pip install numpy pandas jupyter notebook openpyxl xlrd
import pandas as pd
import os
import openpyxl
import xlrd
import numpy as np

In [2]:
# IMPORT Tổng hợp nhập xuất tồn -> RENAME 
Data_Ton = pd.read_excel("Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A,F,K:L",skipfooter=1,header=5)
Data_Ton=Data_Ton.rename(columns={"Unnamed: 0":"ID","Unnamed: 5":"ID_Category","Số lượng.2":"Tổng xuất", "Giá trị.2":"Giá trị xuất" })

#Import Quotation
Data_Request=pd.read_excel("Request Quotation.xlsx",usecols="B:H")

#Import Báo cáo giá bán tồn kho
Data_GiaBanTonKho=pd.read_excel("Báo cáo giá bán, tồn kho.xls",usecols="C,F,J",header=4,skipfooter=1)
Data_GiaBanTonKho=Data_GiaBanTonKho.rename(columns={"Mã số":"ID"})
Data_GiaBanTonKho.dropna(how='all',inplace=True)


#Import Sổ chi tiết bán hàng
Data_BanHang=pd.read_excel("Sổ chi tiết bán hàng.xls",usecols="F,M",skipfooter=1,header=6)
Data_BanHang=Data_BanHang.rename(columns={"Unnamed: 0":"Ngày bán","Unnamed: 5":"ID" ,"Giá bán":"Giá bán giảm"})


#Import Sổ chi tiết mua hàng
Data_MuaHang=pd.read_excel("Sổ chi tiết mua hàng (2).xls",usecols="A,C,E:F,I",header=5,skipfooter=1)
Data_MuaHang=Data_MuaHang.rename(columns={"Unnamed: 0":"Ngày nhập",
                                        "Unnamed: 2":"Nhà cung cấp",
                                        "Unnamed: 4":"Mã nhóm hàng",
                                        "Unnamed: 5":"ID",
                                        "Số lượng":"Số lượng nhập"})


In [3]:
# Calcuation on Tong Hop Ton Kho -------------------------------------------

# Gia Von= Gia Tri Xuat/SL Xuat * 1.05
Data_Ton['Giá vốn']=np.around(Data_Ton['Giá trị xuất']/Data_Ton['Tổng xuất']*1.05,0)


In [4]:
# Merge Request -> Gia Von
Data_Request= pd.merge(Data_Request,Data_Ton[['ID','Giá vốn']],on='ID', how='left')
# Merge Giá bán (Báo cáo nhập xuất tồn)
Data_Request=pd.merge(Data_Request,Data_GiaBanTonKho[['ID','Giá bán','Tổng tồn']],how='left',on='ID')
# Merge Giá bán hàng
Data_Request=pd.merge(Data_Request,Data_BanHang,how='left',on='ID')
# Merge Giá mua hàng
Data_Request=pd.merge(Data_Request,Data_MuaHang,how='left',on='ID')

In [6]:
# Calcuation on Request -------------------------------------------

#VND/US=24,000; he so = 1.13
Data_Request['Unit Price VND'] = np.around(Data_Request['Unit price']*24000*1.13,0)
Data_Request['Giá vốn chênh lệch %'] = np.around((Data_Request['Unit Price VND']-Data_Request['Giá vốn'])/Data_Request['Giá vốn']*100,2)
#SHOW KET QUA
Data_Request=Data_Request.iloc[:,[0,1,2,3,4,5,6,15,7,16,8,9,10,11,12,13,14]]

In [7]:
Data_Request

,ID,ID Item,Description,PCS/PACK,Min qty,Qty request,Unit price,Mã nhóm hàng,Unit Price VND,Số lượng nhập,Giá vốn,Giá vốn chênh lệch %,Giá bán,Giá bán giảm,Số lượng tồn,Ngày nhập,Nhà cung cấp
0,T24NC50WB,48655-0D140,"Low. arm bushing VIOS NCP150,YARIS NSP152,SIEN...",4,72,3000,6.03,CAOSU,163534.0,32.0,135267.0,20.9,188000.0,170000.0,NaN,2022-04-25,NCC-THAILAND
1,T24NC50WB,48655-0D140,"Low. arm bushing VIOS NCP150,YARIS NSP152,SIEN...",4,72,3000,6.03,CAOSU,163534.0,100.0,135267.0,20.9,188000.0,170000.0,NaN,2022-05-13,NK-CHI
2,T24NC50WB,48655-0D140,"Low. arm bushing VIOS NCP150,YARIS NSP152,SIEN...",4,72,3000,6.03,CAOSU,163534.0,32.0,135267.0,20.9,188000.0,167000.0,NaN,2022-04-25,NCC-THAILAND
3,T24NC50WB,48655-0D140,"Low. arm bushing VIOS NCP150,YARIS NSP152,SIEN...",4,72,3000,6.03,CAOSU,163534.0,100.0,135267.0,20.9,188000.0,167000.0,NaN,2022-05-13,NK-CHI
4,T24NC50WB,48655-0D140,"Low. arm bushing VIOS NCP150,YARIS NSP152,SIEN...",4,72,3000,6.03,CAOSU,163534.0,32.0,135267.0,20.9,188000.0,180000.0,NaN,2022-04-25,NCC-THAILAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,T24AVG02WR,48068-0K010-ARM,"Control Arm VIGO (2WD) LOW,RH",1,1,10,94.17,NaN,2553890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
399,D114XS,UH74 39 33X,"Rear engine mounting FIGHTER,RANGER SMALL",4,48,20,3.66,NaN,99259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
400,D30470,UH71 33 651,"Cylinder piston FIGHTER,RANGER",1,60,30,8.32,NaN,225638.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
401,O24314WS,51392-T5A-J01,"Low. arm bushing CITY (2014),JAZZ (2015),HR-V ...",8,144,104,3.47,NaN,94106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [6]:

# EXPORT FILE DATA
with pd.ExcelWriter('Report Quotation Request.xlsx') as writer:
    Data_Request.to_excel(writer, sheet_name='Data')
  

